# Manual Testing

See `readme.md` for setup instructions.

Then make sure this jupyter notebook is restarted before rebuilding, or `maturin` will fail to copy the fresh `.pyd` library into `.env/Lib/megalinref/` .

In [1]:
import json
import megalinref as mlr
import pandas as pd
import numpy as np
import util.unit_conversion as uc
import math

import functools
import operator

In [3]:
if False:
    # Fetch new Data
    xx = mlr.download_fresh_data_as_json()
    with open("./temporary_test_data/road_network.json","w") as f:
        json.dump(xx, f)
else:
    # Use cached data
    with open("./temporary_test_data/road_network.json","r") as f:
        xx = json.load(f)

In [4]:
# typical 1.5s - 4.7s
slk_lookup = mlr.SLKLookup(xx)

In [4]:
del xx

In [4]:
any_carriageway = functools.reduce(operator.or_, mlr.Cwy.values())
any_carriageway

7

In [5]:
any_network_type = functools.reduce(operator.or_, mlr.NetworkType.values())
any_network_type

63

In [6]:
result = slk_lookup.lookup(-32.0832347, 115.8741531, any_carriageway, any_network_type)
result

{'feature': {'ROAD': 'H018',
  'CWY': 'Right',
  'START_SLK': 11.84,
  'END_SLK': 12.46,
  'START_TRUE_DIST': 2.19,
  'END_TRUE_DIST': 2.81,
  'NETWORK_TYPE': 'State Road'},
 'slk': 12.377082008066086,
 'true': 2.727082008066085,
 'distance_metres': 35.053266696990825}

In [7]:
# should be Mitchell Fwy, left lane at slk 10
lat_lon = (-31.89006203575722, 115.80183730752809)
expected_result = {
    "feature":{
        "ROAD": "H016",
        "CWY": "Left",
    },
    "slk":10
}

result = slk_lookup.lookup(*lat_lon, any_carriageway, any_network_type)

assert all(result["feature"][item] == expected_result["feature"][item] for item in ["ROAD","CWY"])
assert math.isclose(result["slk"], expected_result["slk"], abs_tol=0.01)

In [14]:
# should be Mitchell Fwy, left lane at slk 10
lat_lon = (-31.89006203575722, 115.80183730752809)
expected_result = {
    "feature":{
        "ROAD": "H016",
        "CWY": "Left",
    },
    "slk":10
}

bincode = slk_lookup.to_binary()
with open("./temporary_test_data/binary_cache.bin", "wb") as f:
    f.write(bincode)

with open("./temporary_test_data/binary_cache.bin", "rb") as f:
    bincode = f.read()

slk_lookup_reloaded = mlr.SLKLookup.from_binary(bincode)
result = slk_lookup_reloaded.lookup(*lat_lon, any_carriageway, any_network_type)
assert all(result["feature"][item] == expected_result["feature"][item] for item in ["ROAD","CWY"])
assert math.isclose(result["slk"], expected_result["slk"], abs_tol=0.01)

In [2]:
with open("./temporary_test_data/binary_cache.bin", "rb") as f:
    bincode = f.read()

slk_lookup_reloaded = mlr.SLKLookup.from_binary(bincode)

In [8]:
slk_lookup.lookup(
  lat=-31.89006203575722,
  lon=115.80183730752809,
  cwy=mlr.Cwy["L"] | mlr.Cwy["R"],
  network_type=mlr.NetworkType["State Road"] | mlr.NetworkType["Local Road"] 
)

{'feature': {'ROAD': 'H016',
  'CWY': 'Left',
  'START_SLK': 9.84,
  'END_SLK': 10.68,
  'START_TRUE_DIST': 9.84,
  'END_TRUE_DIST': 10.68,
  'NETWORK_TYPE': 'State Road'},
 'slk': 9.99999981522603,
 'true': 9.99999981522603,
 'distance_metres': 1.1852499613658425e-09}

In [10]:
type(bb)

bytes

In [11]:
f"{len(bb)/1024/1024:.2f} MB"

'32.70 MB'

In [12]:
import json
f"{len(json.dumps(xx))/1024/1024:.2f} MB"

'98.85 MB'